In [25]:
import gym
import pandas as pd

env = gym.make("CartPole-v1", render_mode="human")
env.action_space.seed(42)
observation, info = env.reset(seed=42)
score = 0

df = pd.read_csv(r'C:\Users\pieni\Documents\GitHub\CartPole\observation_space.csv', dtype=str)
df2 = pd.read_csv(r'C:\Users\pieni\Documents\GitHub\CartPole\ruleset.csv', dtype=str)

observations = df.values.tolist()
rulesets = df2.values.tolist()

list_rulesets = []

for ruleset in rulesets:
    dict = {}
    rule = "".join(ruleset)
    for i in range(len(observations)):
        dict[observations[i][0]] = rule[i]
    list_rulesets.append(dict)

def action(observation, ruleset):
    cart_position = observation[0]
    cart_velocity = observation[1]
    pole_angle = observation[2]
    pole_angular_velocity = observation[3]

    cart_position = 1 if cart_position >= 0 else 0
    cart_velocity = 1 if cart_velocity >= 0 else 0
    pole_angle = 1 if pole_angle >= 0 else 0
    pole_angular_velocity = 1 if pole_angular_velocity >= 0 else 0

    key = str(cart_position) + str(cart_velocity) + str(pole_angle) + str(pole_angular_velocity)
    return int(ruleset[key])

for ruleset in list_rulesets[:5]:
    maxscore = 0
    print('Ruleset: ', ruleset)
    for episode in range(100):
        observation, reward, terminated, truncated, info = env.step(action(observation, ruleset))
        score += reward
        
        if terminated or truncated:
            observation, info = env.reset()
            if score > maxscore:
                maxscore = score 
            score = 0
    print("Max Score: ", maxscore)

env.close()

Ruleset:  {'0000': '0', '0001': '0', '0010': '0', '0011': '0', '0100': '0', '0101': '0', '0110': '0', '0111': '0', '1000': '0', '1001': '0', '1010': '0', '1011': '0', '1100': '0', '1101': '0', '1110': '0', '1111': '0'}
Max Score:  10.0
Ruleset:  {'0000': '0', '0001': '0', '0010': '0', '0011': '0', '0100': '0', '0101': '0', '0110': '0', '0111': '0', '1000': '0', '1001': '0', '1010': '0', '1011': '0', '1100': '0', '1101': '0', '1110': '0', '1111': '1'}
Max Score:  21.0
Ruleset:  {'0000': '0', '0001': '0', '0010': '0', '0011': '0', '0100': '0', '0101': '0', '0110': '0', '0111': '0', '1000': '0', '1001': '0', '1010': '0', '1011': '0', '1100': '0', '1101': '0', '1110': '1', '1111': '0'}
Max Score:  10.0
Ruleset:  {'0000': '0', '0001': '0', '0010': '0', '0011': '0', '0100': '0', '0101': '0', '0110': '0', '0111': '0', '1000': '0', '1001': '0', '1010': '0', '1011': '0', '1100': '0', '1101': '0', '1110': '1', '1111': '1'}
Max Score:  10.0
Ruleset:  {'0000': '0', '0001': '0', '0010': '0', '0011'